In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

In [3]:
def getDataAndLabel(df):
    label = np.clip(df['item_cnt_month'], 0, 20)
    data = df.drop(columns=['item_cnt_month'])
    return (data, label)

In [4]:
def create_submission(tset, testdf, preds):
    tset['item_cnt_month'] = preds
    sub = testdf.merge(tset, 'inner', on=['shop_id', 'item_id']).loc[:,['ID', 'item_cnt_month']]
    sub.to_csv('./submission.csv', index=False)

In [5]:
def fi(model, df):
    f = model.feature_importances_
    cols = df.columns
    x = pd.DataFrame({'Features': cols, 'Importance': f})
    x.sort_values(by='Importance', ascending=False, ignore_index=True, inplace=True)
    return x

In [6]:
def train_rf_regressor(X, y, 
                       n_estimators=100, max_samples=5000000,
                       min_samples_leaf=2, oob_score=True,
                       n_jobs=-1, verbose=2):
    return RandomForestRegressor(n_estimators=n_estimators, 
                                 max_samples=max_samples, 
                                 min_samples_leaf=min_samples_leaf,
                                 oob_score=oob_score, n_jobs=-1, 
                                 verbose=verbose).fit(X,y)

In [7]:
def validate_rf_regressor(model, valid_x, valid_y):
    print(f'Out of bag score is {model.oob_score_}')
    pred = np.clip(model.predict(valid_x), 0, 20)
    rmse_score = np.sqrt(mean_squared_error(valid_y, pred))
    print(f'RMSE score on validation set is {rmse_score}')

## Read master data file

In [8]:
train_master = pd.read_pickle('./input/train_2.pkl')

In [9]:
train_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35129600 entries, 3 to 38422999
Data columns (total 14 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   shop_id                 int8   
 1   item_id                 int16  
 2   date_block_num          int8   
 3   month                   int8   
 4   item_cnt_month          float64
 5   item_cat                int8   
 6   item_cnt_minus_1        float64
 7   item_cnt_minus_2        float64
 8   item_cnt_minus_3        float64
 9   mnth_med_price_minus_1  float64
 10  mnth_med_price_minus_2  float64
 11  mnth_med_price_minus_3  float64
 12  item_cnt_mean           float64
 13  mnth_med_price_mean     float64
dtypes: float64(9), int16(1), int8(4)
memory usage: 2.8 GB


## Read Test dataset from Kaggle

In [10]:
test = pd.read_csv('./input/test.csv')

## Train Random forest regressor and see baseline performance

### Declare training and validation sets

In [11]:
mask_test = train_master.date_block_num ==34

In [12]:
xset = train_master[~mask_test].copy()
test_set = train_master[mask_test].copy()

## Split xs into train set and valid set to calculate validation score 

In [13]:
xs = xset.loc[~(xset.date_block_num==33)]
valid_x = xset.loc[(xset.date_block_num==33)]

## Split train and valid sets into data and lables

In [14]:
xs, y = getDataAndLabel(xs)
valid_x, valid_y = getDataAndLabel(valid_x)

In [18]:
xs.shape[0]*0.2

6586800.0

## Random forest regressor

In [19]:
rfr = train_rf_regressor(xs, y, n_estimators=50, max_samples=6000000)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 50building tree 2 of 50
building tree 3 of 50
building tree 4 of 50

building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 20.9min


building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of 50
building tree 46 of 50
building tree 47 of 50
building tree 48 of 50
building tree 49 of 50
building tree 50 of 50


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 29.7min finished


In [20]:
validate_rf_regressor(rfr, valid_x, valid_y)

Out of bag score is 0.5465215442575266


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    2.3s


RMSE score on validation set is 0.4380863392365203


[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    3.2s finished


In [21]:
fi(rfr, xs)

,Features,Importance
0,item_cnt_minus_1,0.354792
1,item_id,0.200114
2,item_cnt_mean,0.099201
3,shop_id,0.081390
4,date_block_num,0.063077
5,month,0.047653
6,item_cat,0.037681
7,mnth_med_price_mean,0.037327
8,mnth_med_price_minus_1,0.022946
9,item_cnt_minus_3,0.015890


In [23]:
rfr_2 = train_rf_regressor(xs, y, n_estimators=50, max_samples=15000000)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 50building tree 2 of 50
building tree 3 of 50
building tree 4 of 50

building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50building tree 23 of 50

building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 45.1min


building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of 50
building tree 46 of 50
building tree 47 of 50
building tree 48 of 50
building tree 49 of 50
building tree 50 of 50


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 61.4min finished


In [24]:
validate_rf_regressor(rfr_2, valid_x, valid_y)

Out of bag score is 0.5526661329196076


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    3.3s


RMSE score on validation set is 0.43720813496267724


[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    4.5s finished


In [25]:
rfr_3 = train_rf_regressor(xs, y, n_estimators=100, max_samples=6000000)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100building tree 4 of 100

building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100building tree 28 of 100

building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 21.8min


building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100building tree 68 of 100

building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 57.9min finished


In [26]:
validate_rf_regressor(rfr_3, valid_x, valid_y)

Out of bag score is 0.5507886558840603


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    2.9s


RMSE score on validation set is 0.4334907188001643


[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    8.4s finished
